In [ ]:
import torch
import gc

torch.cuda.empty_cache()  # Clears cached memory
gc.collect()



0

In [ ]:


import numpy as np
print(np.__version__)


import torch
print(torch.__version__)

import torch.nn as nn
import torch.nn.functional as F

torch.manual_seed(0)

from tqdm.notebook import trange

import random
import math
torch.manual_seed(0)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
from tqdm import trange
import torch
import gc


2.1.2
2.5.1+cu121


In [13]:
class ConnectFour:
    def __init__(self):
        self.row_count = 6
        self.column_count = 7
        self.action_size = self.column_count
        self.in_a_row = 4
        
    def __repr__(self):
        return "ConnectFour"
        
    def get_initial_state(self):
        return np.zeros((self.row_count, self.column_count))
    
    def get_next_state(self, state, action, player):
        row = np.max(np.where(state[:, action] == 0))
        state[row, action] = player
        return state
    
    def get_valid_moves(self, state):
        return (state[0] == 0).astype(np.uint8)
    
    def check_win(self, state, action):
        if action == None:
            return False
        
        row = np.min(np.where(state[:, action] != 0))
        column = action
        player = state[row][column]

        def count(offset_row, offset_column):
            for i in range(1, self.in_a_row):
                r = row + offset_row * i
                c = action + offset_column * i
                if (
                    r < 0 
                    or r >= self.row_count
                    or c < 0 
                    or c >= self.column_count
                    or state[r][c] != player
                ):
                    return i - 1
            return self.in_a_row - 1

        return (
            count(1, 0) >= self.in_a_row - 1 # vertical
            or (count(0, 1) + count(0, -1)) >= self.in_a_row - 1 # horizontal
            or (count(1, 1) + count(-1, -1)) >= self.in_a_row - 1 # top left diagonal
            or (count(1, -1) + count(-1, 1)) >= self.in_a_row - 1 # top right diagonal
        )
    
    def get_value_and_terminated(self, state, action):
        if self.check_win(state, action):
            return 1, True
        if np.sum(self.get_valid_moves(state)) == 0:
            return 0, True
        return 0, False
    
    def get_opponent(self, player):
        return -player
    
    def get_opponent_value(self, value):
        return -value
    
    def change_perspective(self, state, player):
        return state * player
    
    def get_encoded_state(self, state):
        encoded_state = np.stack(
            (state == -1, state == 0, state == 1)
        ).astype(np.float32)
        
        if len(state.shape) == 3:
            encoded_state = np.swapaxes(encoded_state, 0, 1)
        
        return encoded_state

In [14]:
class ResNet(nn.Module):

    def __init__(self,game,num_resBlocks,num_hidden, in_channels):
        super().__init__()
        #input dimension is : 100x3x6x7 
        #kernel = convolutional matrix 
        #num_hidden = number of filters
        #kernel_size 3x3
        self.num_resBlocks = num_resBlocks
        self.in_channels = in_channels #this is the channel size of the matrice, the reason this is  3 because the cases where player 1 plays , where player 2 plays and nobody plays is represented by 3 different layers where each is represented by 1 and the rest being 0 for connect 4. 
        self.out_channels = num_hidden # channel size of the output not the width and length  
        self.startBlock = nn.Sequential(
            nn.Conv2d(self.in_channels,self.out_channels,kernel_size=3,padding=1), #3 6 7 -> num_hidden 6 7
            nn.BatchNorm2d(self.out_channels),
            nn.ReLU()
        )
        self.BackBone = nn.ModuleList([ResBlock(self.out_channels) for i in range(self.num_resBlocks)] ) #We use modulelist instead of sequential because it allows us to use skip connections 
        self.value_head = nn.Sequential(
            nn.Conv2d(self.out_channels, 3, kernel_size=3, padding=1), #DONT KNOW WHY 32 yet may be empirical
            nn.BatchNorm2d(3),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(3*game.row_count*game.column_count, 1),
            nn.Tanh()  #value is between -1 and 1
        )
        self.policy_head = nn.Sequential(
            nn.Conv2d(self.out_channels, 32, kernel_size=3, padding=1), #DONT KNOW WHY 32 yet 
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(32*game.row_count*game.column_count, game.action_size), #the action size for each game may be different 
            
        )
        self.to(device)
   

    def forward(self,state): 
        
        x = self.startBlock(state)
        for block in self.BackBone:
            x = block(x)  # Apply each ResBlock manually
        policy = self.policy_head(x)
        value = self.value_head(x)
        return policy, value

        

class ResBlock(nn.Module):
    def __init__(self,out_channels):
        super().__init__()
        self.out_channels = out_channels
        self.Conv2d_1 = nn.Conv2d(self.out_channels,self.out_channels,kernel_size=3,padding=1)
        self.BatchNorm_1 = nn.BatchNorm2d(self.out_channels)
        self.Conv2d_2 = nn.Conv2d(self.out_channels,self.out_channels,kernel_size=3,padding=1)
        self.BatchNorm_2 = nn.BatchNorm2d(self.out_channels)

    def forward(self,x):
        temp = x
        x = self.BatchNorm_2(self.Conv2d_2(F.relu(self.BatchNorm_1(self.Conv2d_1(x)))))
        x = temp + x
        x = F.relu(x)
        return x 

        


In [15]:
class Node:
     def __init__(self,state,args,transition_action, player,game):
          self.state = state 
          self.nb_visits = 1
          self.value  = 0 
          self.prior_proba = 0
          self.children = []
          self.parent  = None 
          self.args = args      
          self.transition_action = transition_action
          self.player = player 
          self.game = game


     def get_prior_proba(self):
          return self.prior_proba
     def get_player(self):
          return self.player
     def get_transition_action(self):
          return self.transition_action

     def get_visits(self):
          return self.nb_visits
     def get_state(self):
          return self.state
     def set_value(self, value ):
          self.value = value

     def get_value(self):
          return self.value
     def set_prior_proba(self,proba):
          self.prior_proba = proba

     def get_children(self):
          return self.children

     def add_child(self, child_node,transition_action):
          self.children.append((child_node,transition_action))
          child_node.set_parent(self)
          
     def set_parent(self, parent_node):
          self.parent = parent_node

     def get_parent(self):
          return self.parent 

     def calc_PUCT(self):
          parent = self.get_parent() 
          parent_visits = parent.get_visits()
          if self.nb_visits == 0:
            q_value = 0
          else:
               q_value = 1 - ((self.value / self.nb_visits) + 1) / 2
          res = q_value + self.args['C'] * (math.sqrt(parent_visits) / (self.nb_visits + 1)) *  self.prior_proba
          #print(f"res with nb_visits = ",self.nb_visits, " and parent_visits = ", parent_visits, " is ", res)
          return res

     def choose_best_child(self):
          
          if self.children != []:
               best_tuple = max(self.children, key=lambda item: item[0].calc_PUCT()) #we choose the best child node based on the highest PUCT value
               best_child = best_tuple[0]
               best_action = best_tuple[1]

               return best_child, best_action
          else :
              return None, None


     def increment_visit(self):
          self.nb_visits += 1 

     def add_value(self, value):
          self.value += value 
     def selfPlay_policy(self):
          action_size = 7
          total_visits = sum(child[0].get_visits() for child in self.children) + 1e-8
          policy = [0] * action_size

          for child_node, _ in self.children:
               action = child_node.get_transition_action()
               if action is not None:
                    # Use += in case there are multiple child nodes for the same action (edge case)
                    policy[action] += child_node.get_visits() / total_visits

          return policy

     def is_fully_expanded(self):
          if self.children == []:
               return False
          else:
               return True
                    
     def expand(self,probas):
          for i, proba in enumerate(probas) :  
                #either we get all the actions or the all allowed actions I am not sure but the dimensions of the policy returned by the ResNet must equal the number of actions / states 
                if proba > 0 : 
                    child_state = self.state.copy()
                    child_state = self.game.get_next_state(child_state, i, 1)
                    child_state = self.game.change_perspective(child_state, player=-1)#dont forget to change the perspective every time a new node is added
                    child_node = Node(child_state,self.args,i,self.player*-1,self.game)
                    child_node.set_prior_proba(proba)
                    child_node.set_parent(self)
                    self.add_child(child_node,i)


     def backpropagate(self, value):
        self.value += value
        self.nb_visits += 1
        
        value = self.game.get_opponent_value(value)
        if self.parent is not None:
            self.parent.backpropagate(value)  

In [22]:
import time
class MCTSParallel:

    def __init__(self,game,args,model):
        self.game =game 
        self.args = args
        self.model=model 


    @torch.no_grad() # EXTREMELY IMPORTANT IF NOT THE GPU MEMORY FILLS UP
    def search(self,states,spGames,player):
            
        start_time = time.time()
        policies, values =  self.model.forward(torch.tensor(self.game.get_encoded_state(states), dtype=torch.float32, device=device))
        policies = torch.softmax(policies, axis=1).cpu().numpy()
        policies = (1 - self.args['dirichlet_epsilon']) * policies + self.args['dirichlet_epsilon'] \
        * np.random.dirichlet([self.args['dirichlet_alpha']] * self.game.action_size, size=policies.shape[0])
        #for the current root state we are in in every game get its policy
        valid_moves = np.array([self.game.get_valid_moves(state) for state in states])
        #print("len(states) = ", len(states))
        #print("len(spGames) = ", len(spGames))
        for i in range(len(states)):
            spGames[i].root = Node(states[i],self.args,None,player,self.game) #we create a new root node for each game and set its visit count to 1
            policy = policies[i] * valid_moves[i] #we multiply the policy by the valid moves to get the valid policy
            policy /= np.sum(policy) #we normalize thestates,spGames policy to get the probabilities of each action
            spGames[i].root.expand(policy) # we only add the dirichlet noise at the root node


        for _ in range(self.args["MCTS_iterations"]):
            for i in range(len(spGames)):
                spGames[i].node = None
                leaf_node = spGames[i].root
                while leaf_node.is_fully_expanded():
                    leaf_node, action_taken_by_opponent = leaf_node.choose_best_child() 
                #we go down the tree until we reach a leaf of the root node of the spgame
                value, is_terminal = self.game.get_value_and_terminated(leaf_node.get_state(), leaf_node.transition_action)
                value = self.game.get_opponent_value(value) 
                if is_terminal:
                    leaf_node.set_value(value)
                    leaf_node.backpropagate(value)
                    spGames[i].node = None
                else: 
                    spGames[i].node = leaf_node
                #if we achieved a terminal state we backpropagate and this spgame is done if not we set spg.node to the leaf node to indicate that the game isnt over

            not_finished_games =  [spGame for spGame in spGames if spGame.node != None]
            #we get the not over games
            if not_finished_games  == []:
                continue 
            rollout_states = torch.tensor(
                np.array([self.game.get_encoded_state(spGame.node.get_state()) for spGame in not_finished_games]),
                dtype=torch.float32,
                device=device  
            )

            policies, values = self.model.forward(rollout_states)
            policies = torch.softmax(policies, axis=1).cpu().numpy()
            values = values.cpu().numpy()
            for i in range(len(not_finished_games)):
                valid_moves  = self.game.get_valid_moves(not_finished_games[i].node.get_state())
                true_policy = policies[i] * valid_moves
                true_policy /= np.sum(true_policy) 
                value = values[i]
                not_finished_games[i].node.set_value(value)
                not_finished_games[i].node.backpropagate(value)
                not_finished_games[i].node.expand(true_policy)
            
        duration = time.time() - start_time
        
         
         
         
         
        



    

In [23]:
def evaluate_vs_random(model, game, num_games=10):
    
    wins, draws, losses = 0, 0, 0

    for _ in range(num_games):
        state = game.get_initial_state()
        player = 1
        while True:
            if player == 1:
                # Model's turn
                encoded = torch.tensor(game.get_encoded_state(state), dtype=torch.float32, device=device).unsqueeze(0)
                with torch.no_grad():
                    policy_logits, _ = model(encoded)
                    policy = torch.softmax(policy_logits, dim=1).cpu().numpy()[0]
                    valid_moves = game.get_valid_moves(state)
                    policy *= valid_moves
                    if policy.sum() == 0:
                        policy = valid_moves / valid_moves.sum()
                    else:
                        policy /= policy.sum()
                    action = np.random.choice(game.action_size, p=policy)

            else:
                # Random agent's turn
                valid_moves = game.get_valid_moves(state)
                action = np.random.choice(np.flatnonzero(valid_moves))

            state = game.get_next_state(state, action, player)
            value, is_terminal = game.get_value_and_terminated(state, action)
            if is_terminal:
                if value == 1:
                    if player == 1:
                        wins += 1
                    else:
                        losses += 1
                else:
                    draws += 1
                break
            player = game.get_opponent(player)

    total = wins + draws + losses
    print(f"Model vs Random — Wins: {wins}, Draws: {draws}, Losses: {losses} ({total} games)")


In [24]:

class AlphaZeroParallel():



    def __init__(self,game,args,model,optimizer):
        self.game = game
        self.model = model
        self.args = args 
        self.replay_buffer = [] #we store the historical data for the neural network valeus to be corrected here 
        self.current_game_tree =  SPG(game,args)
        self.optimizer = optimizer
        self.MCTS = MCTSParallel(self.game,self.args,self.model)
        self.value_losses = []
        self.policy_losses = []
        self.iterations = [0]
    def game_of_SelfPlay(self,spGames): 
            return_memory = []
            player  = 1
            action = None
            
            self.model.eval()
            to_remove = []
            while len(spGames) > 0: 
                states = np.stack([spg.state for spg in spGames])
                neutral_states = self.game.change_perspective(states, player)
                self.MCTS.search(neutral_states,spGames,player)
                
                
                for i in range(len(spGames))[::-1]:
                    spg = spGames[i]
                      
                    single_policy = np.zeros(self.game.action_size)
                    for child,action in spg.root.get_children():
                        single_policy[action] = child.nb_visits
                    single_policy /= np.sum(single_policy)


                    
                    

                    true_policy = single_policy
                    spGames[i].memory.append((spg.root.state,true_policy,player))

                    #With high temperature: probabilities become more uniform
                    #With low temperature: differences get exaggerated, making the highest probability action even more likely
                    temperature_action_probs = single_policy ** (1 / self.args['temperature'])
                    temperature_action_probs /= np.sum(temperature_action_probs)
                    action = np.random.choice(len(temperature_action_probs), p=temperature_action_probs)
                    spg.state = self.game.get_next_state(spg.state, action, player)
                    
                    game_value, is_terminal = self.game.get_value_and_terminated(spg.state, action)
                    #once an spg is finished we store the data that we stored in spg.memory in the replay buffer with the value of the end game and we remove the spg from the list
                    #no need to keep track of the node tree since every state is already stored inside the spg memory 
                    if is_terminal:
                         #seems like we are either putting 1 or 0 as the value of the position instead of a non discrete value
                         # Over time, the value target for a position becomes an average of many discrete outcomes, which can be a non-discrete float

                        
                        for hist_state,true_policy,hist_player in spg.memory:

                            hist_outcome = game_value if hist_player == player else self.game.get_opponent_value(game_value)


                            return_memory.append((
                            self.game.get_encoded_state(hist_state),
                            true_policy,
                            hist_outcome))
                        del spGames[i]
                       
                player = self.game.get_opponent(player)
            return return_memory 


    def train(self,memory):
            random.shuffle(memory)
            for j in trange(0, len(memory),self.args["batch_size"], desc="Training", leave=False):
                batch = memory[j:min(j+self.args["batch_size"], len(memory))]
                states, true_policies, values = zip(*batch)
                states, true_policies, values = np.array(states), np.array(true_policies), np.array(values)
                states = torch.tensor(states, dtype=torch.float32, device=device)
                policy_targets = torch.tensor(true_policies, dtype=torch.float32, device=device)
                value_targets = torch.tensor(values, dtype=torch.float32, device=device).view(-1, 1)
                policy_logits, value_logits = self.model.forward(states)

                policy_loss = F.cross_entropy(policy_logits,policy_targets)

                value_loss = F.mse_loss(value_logits,value_targets)
                loss = policy_loss + value_loss
                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()

                print(f"value_loss : {value_loss}, policy_loss : {policy_loss}, loss : {loss}")
                self.value_losses.append(value_loss.item())
                self.policy_losses.append(policy_loss.item())   
                self.iterations.append(self.iterations[-1] + 1)


    def learn(self):   

        
        if self.args["load_model"] == True:
            self.model.load_state_dict(torch.load(f"model_{self.game}.pt"))
            self.optimizer.load_state_dict(torch.load(f"optimizer_{self.game}.pt"))   
        for  i in range(self.args['iterations']):
            memory = []
            self.model.eval()
            for i in trange(self.args['num_parallel_selfplays']):
                start_time = time.time()
                spGames = [SPG(self.game,self.args) for h in range(self.args["parallel_games"])]     
                memory += self.game_of_SelfPlay(spGames)
                end_time = time.time()
                print(f"Self-play time: {end_time - start_time:.4f} seconds") 

            
            self.model.train()
            
            
            for epoch in trange(self.args['num_epochs']): #without training ont he same data the value loss is very high 
                start_time = time.time()
                self.train(memory)
                end_time = time.time()
                print(f"Training time: {end_time - start_time:.4f} seconds")
            self.replay_buffer = []
            if i % self.args["save_frequency"] == 0:
                #self.plot_metrics()
                torch.save(self.model.state_dict(), f"model_{self.game}.pt")
                torch.save(self.optimizer.state_dict(), f"optimizer_{self.game}.pt")

            
        

    def plot_metrics(self, window=10):
        import matplotlib.pyplot as plt
        import numpy as np

        def moving_average(data, window=10):
            if len(data) < window:
                return data  # Not enough data to smooth
            return np.convolve(data, np.ones(window)/window, mode='valid')

        plt.figure(figsize=(12, 8))

        # Raw losses
        plt.subplot(2, 1, 1)
        plt.plot(self.iterations, self.value_losses, label='Value Loss (Raw)', alpha=0.3)
        plt.plot(self.iterations, self.policy_losses, label='Policy Loss (Raw)', alpha=0.3)

        # Smoothed losses
        if len(self.value_losses) >= window:
            smoothed_iter = self.iterations[window-1:]
            plt.plot(smoothed_iter, moving_average(self.value_losses, window), label='Value Loss (Smoothed)', linewidth=2)
            plt.plot(smoothed_iter, moving_average(self.policy_losses, window), label='Policy Loss (Smoothed)', linewidth=2)

        plt.title('AlphaZero Training Losses')
        plt.xlabel('Training Step')
        plt.ylabel('Loss')
        plt.legend()
        plt.grid(True)

        plt.tight_layout()
        plt.savefig(f'alphazero_metrics_{len(self.iterations)}.png')
        plt.close()

            


        
    def free_memory(self):
        torch.cuda.empty_cache()
        gc.collect()
class SPG():
    
    def __init__(self,game,args):
        self.game = game
        self.memory = []
        self.state = game.get_initial_state()
        self.root = None
        self.node = None
        


         



In [25]:
#Testing the ResNet model

game = ConnectFour()
model = ResNet(game, 9, 128, 3).to(device)

# Generate a dummy board
state = game.get_initial_state()
encoded = torch.tensor(game.get_encoded_state(state), dtype=torch.float32, device=device).unsqueeze(0)

# Run a forward pass
policy_logits, value = model(encoded)
policy_probs = torch.softmax(policy_logits, dim=1)

print("Policy shape:", policy_probs.shape)
print("Value prediction:", value.item())
print("Policy probs (sum):", policy_probs.sum().item())
print("Top move prob:", policy_probs.max().item())


Policy shape: torch.Size([1, 7])
Value prediction: 0.31358757615089417
Policy probs (sum): 0.9999999403953552
Top move prob: 0.2724999785423279


In [28]:
#Testing a round of SelfPlay
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=0.0001)

args = {
    'C': 2,
    'MCTS_iterations': 50,
    'num_parallel_selfplays': 3,
    'iterations': 8,
    'parallel_games': 50,
    'batch_size': 128,
    'dirichlet_epsilon': 0.25,
    'dirichlet_alpha': 0.3,
    'temperature': 1.25,
    'load_model': False,
    'save_frequency': 1,
    'num_epochs': 4,
}
alphaZero = AlphaZeroParallel(game,args,model,optimizer)
alphaZero.learn()

 33%|███▎      | 1/3 [00:14<00:28, 14.06s/it]

Self-play time: 14.0550 seconds


 67%|██████▋   | 2/3 [00:29<00:14, 14.64s/it]

Self-play time: 15.0443 seconds


100%|██████████| 3/3 [00:42<00:00, 14.16s/it]


...

value_loss : 0.43417835235595703, policy_loss : 1.6651010513305664, loss : 2.0992794036865234
value_loss : 0.37903350591659546, policy_loss : 1.682751178741455, loss : 2.0617847442626953
value_loss : 0.4059610962867737, policy_loss : 1.6685290336608887, loss : 2.0744900703430176
value_loss : 0.4468681514263153, policy_loss : 1.6789638996124268, loss : 2.1258320808410645
value_loss : 0.334755003452301, policy_loss : 1.6981083154678345, loss : 2.0328633785247803
value_loss : 0.3361005485057831, policy_loss : 1.7275558710098267, loss : 2.0636563301086426
value_loss : 0.38244786858558655, policy_loss : 1.6952531337738037, loss : 2.0777010917663574


value_loss : 0.32067131996154785, policy_loss : 1.6665852069854736, loss : 1.9872565269470215
value_loss : 0.31840768456459045, policy_loss : 1.7042536735534668, loss : 2.0226614475250244
value_loss : 0.35943302512168884, policy_loss : 1.728233814239502, loss : 2.0876667499542236
value_loss : 0.36570894718170166, policy_loss : 1.7075841426849365, loss : 2.0732932090759277
value_loss : 0.3665899932384491, policy_loss : 1.6801517009735107, loss : 2.0467417240142822
value_loss : 0.3857433795928955, policy_loss : 1.7262792587280273, loss : 2.112022638320923
value_loss : 0.4364328980445862, policy_loss : 1.663201093673706, loss : 2.0996339321136475
value_loss : 0.42344143986701965, policy_loss : 1.7088251113891602, loss : 2.1322665214538574
value_loss : 0.3928392231464386, policy_loss : 1.7094042301177979, loss : 2.102243423461914


value_loss : 0.33876463770866394, policy_loss : 1.6163431406021118, loss : 1.9551078081130981
value_loss : 0.31361132860183716, policy_loss : 1.695719599723816, loss : 2.009330987930298
value_loss : 0.4049452245235443, policy_loss : 1.6591758728027344, loss : 2.0641210079193115
value_loss : 0.42008349299430847, policy_loss : 1.6871252059936523, loss : 2.107208728790283
value_loss : 0.31793639063835144, policy_loss : 1.695885419845581, loss : 2.013821840286255


100%|██████████| 4/4 [00:01<00:00,  3.33it/s]

value_loss : 0.29116761684417725, policy_loss : 1.6696133613586426, loss : 1.9607809782028198
value_loss : 0.33534157276153564, policy_loss : 1.6873825788497925, loss : 2.022724151611328
Training time: 0.2817 seconds


In [27]:

evaluate_vs_random(model, game, num_games=1000)

Model vs Random — Wins: 707, Draws: 2, Losses: 291 (1000 games)


Model vs Random — Wins: 863, Draws: 2, Losses: 135 (1000 games)
27 minutes of training
lowest value loss: 0.4
lowest policy loss: 1.6

args = {
    'C': 2,
    'MCTS_iterations': 500,
    'num_parallel_selfplays': 8,
    'iterations': 8,
    'parallel_games': 100,
    'batch_size': 128,
    'dirichlet_epsilon': 0.25,
    'dirichlet_alpha': 0.3,
    'temperature': 1.25,
    'load_model': False,
    'save_frequency': 1,
    'num_epochs': 4,
}

No significant difference in total training time, or in training time or selfplay time with the following parameters compared to the original model 
args = {
    'C': 2,
    'MCTS_iterations': 20,
    'num_parallel_selfplays': 2,
    'iterations': 8,
    'parallel_games': 50,
    'batch_size': 64,
    'dirichlet_epsilon': 0.25,
    'dirichlet_alpha': 0.3,
    'temperature': 1.25,
    'load_model': False,
    'save_frequency': 1,
    'num_epochs': 4,
}

In [ ]:
model.load_state_dict(torch.load("model_ConnectFour.pt"))
optimizer.load_state_dict(torch.load("optimizer_ConnectFour.pt")) 

In [29]:
#model.load_state_dict(torch.load("model_ConnectFour.pt"))

import pygame
import numpy as np
import torch
import time

# Game settings
ROWS, COLS = 6, 7
SQUARESIZE = 100
RADIUS = int(SQUARESIZE / 2 - 5)
WIDTH, HEIGHT = COLS * SQUARESIZE, (ROWS + 1) * SQUARESIZE

# Colors
BLACK = (0, 0, 0)
BLUE = (0, 0, 255)
RED = (255, 0, 0)
YELLOW = (255, 255, 0)

def draw_board(screen, board):
    screen.fill(BLACK)
    for c in range(COLS):
        for r in range(ROWS):
            pygame.draw.rect(screen, BLUE, (c*SQUARESIZE, (r+1)*SQUARESIZE, SQUARESIZE, SQUARESIZE))
            piece = board[r][c]
            color = BLACK
            if piece == 1:
                color = RED
            elif piece == -1:
                color = YELLOW
            pygame.draw.circle(screen, color, (int(c*SQUARESIZE + SQUARESIZE/2), int((r+1)*SQUARESIZE + SQUARESIZE/2)), RADIUS)
    pygame.display.update()

def get_column_from_mouse(x):
    return int(x / SQUARESIZE)

def get_row_to_drop(board, col):
    for r in range(ROWS-1, -1, -1):
        if board[r][col] == 0:
            return r
    return None

def play_against_model_pygame(model, game, device, play_as=1):
    pygame.init()
    screen = pygame.display.set_mode((WIDTH, HEIGHT))
    font = pygame.font.SysFont("monospace", 50)
    pygame.display.set_caption("Connect Four - Play vs Model")

    board = np.zeros((ROWS, COLS), dtype=int)
    state = game.get_initial_state()
    player = 1
    game_over = False

    draw_board(screen, board)

    while not game_over:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                return

            if player == play_as and event.type == pygame.MOUSEBUTTONDOWN:
                col = get_column_from_mouse(event.pos[0])
                valid_moves = game.get_valid_moves(state)
                if valid_moves[col] == 1:
                    row = get_row_to_drop(board, col)
                    board[row][col] = player
                    state = game.get_next_state(state, col, player)
                    value, is_terminal = game.get_value_and_terminated(state, col)
                    draw_board(screen, board)

                    if is_terminal:
                        game_over = True
                        text = "You win!" if value == 1 else "Draw!" if value == 0 else "Model wins!"
                        label = font.render(text, True, RED if value == 1 else YELLOW)
                        screen.blit(label, (40, 10))
                        pygame.display.update()
                        time.sleep(3)
                        break

                    player = game.get_opponent(player)

        if player != play_as and not game_over:
            pygame.time.wait(500)
            encoded = torch.tensor(game.get_encoded_state(state), dtype=torch.float32, device=device).unsqueeze(0)
            with torch.no_grad():
                policy_logits, _ = model(encoded)
                policy = torch.softmax(policy_logits, dim=1).cpu().numpy()[0]
                valid_moves = game.get_valid_moves(state)
                policy *= valid_moves
                if policy.sum() == 0:
                    policy = valid_moves / valid_moves.sum()
                else:
                    policy /= policy.sum()
                action = np.random.choice(game.action_size, p=policy)

            row = get_row_to_drop(board, action)
            board[row][action] = player
            state = game.get_next_state(state, action, player)
            value, is_terminal = game.get_value_and_terminated(state, action)
            draw_board(screen, board)

            if is_terminal:
                game_over = True
                text = "Model wins!" if value == 1 else "Draw!" if value == 0 else "You win!"
                label = font.render(text, True, YELLOW if value == 1 else RED)
                screen.blit(label, (40, 10))
                pygame.display.update()
                time.sleep(3)
                break

            player = game.get_opponent(player)

    pygame.quit()



pygame 2.6.1 (SDL 2.28.4, Python 3.12.8)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [30]:
model.eval()
play_against_model_pygame(model, game, device, play_as=1)
